In [1]:
import sys
sys.path.append('/home/joaoc/SimulaCorona/.env/lib/python3.6/site-packages/')

import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import matplotlib.dates as md
from tqdm import tqdm

from paths import *
from scripts import get_data, clean_data

%reload_ext autoreload
%autoreload 2

from scipy.integrate import odeint
import pandas as pd
import numpy as np

In [4]:
# Imports the Google Cloud client library
from google.cloud import storage, bigquery
import tempfile

# Instantiates a client
storage_client = storage.Client.from_service_account_json('../configs/gcloud.json')


In [7]:
config = yaml.load(open('../configs/config.yaml', 'r'))

In [97]:
def get_bucket(config):
    
    storage_client = storage.Client.from_service_account_json('../configs/gcloud.json')
    return storage_client.get_bucket(config['gcloud']['storage']['bucket'])    

def upload_data(df, table_name, filena, config):
    """Saves dataframe as parquet to GCloud storage"""

    bucket = get_bucket(config)
    
    destination_path = f"{config['gcloud']['bigquery']['schema']['prod']}/{table_name}/{filename}"
    
    blob = bucket.blob(destination_path)
        
    with tempfile.TemporaryDirectory() as tmpdirname:
        
        df.to_parquet(tmpdirname + '/temp', index=False)
        source_path = str(tmpdirname + '/temp')
        
        blob.upload_from_filename(source_path)
        
def get_bigquery(path='../configs/gcloud.json'):
    return bigquery.Client.from_service_account_json(path)

def create_table(table_name, config, partition=False):
    
    bigquery_client = get_bigquery()
    
    dataset = bigquery_client.dataset(config['gcloud']['bigquery']['schema']['prod'])
    
    uri = f"gs://{config['gcloud']['storage']['bucket']}/{config['gcloud']['bigquery']['schema']['prod']}/{table_name}/*"
    
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.source_format = bigquery.SourceFormat.PARQUET

    if partition:
        drop_table(table_name, config) # drop table to refresh partitions
        hive = bigquery.external_config.HivePartitioningOptions()
        hive.mode = 'AUTO'
        hive.source_uri_prefix = f"gs://{config['gcloud']['storage']['bucket']}/{config['gcloud']['bigquery']['schema']['prod']}/{table_name}/"
        job_config.hive_partitioning = hive   
    
    load_job = bigquery_client.load_table_from_uri(uri,
                                               dataset.table(table_name),
                                               job_config=job_config)
    load_job.result()
    
def drop_table(table_name, config):
    
    dataset = bigquery_client.dataset(config['gcloud']['bigquery']['schema']['prod'])
    
    bigquery_client.delete_table(dataset.table(table_name), not_found_ok=True)

In [73]:
table_name = 'brasilio'


In [87]:
for i in range(6000):
    filename = f'day={i}/raw'
    df = pd.DataFrame([i], columns=['a'])
    upload_blob(df, table_name, filename, config)

In [99]:
create_table(table_name, config, partition=True)